# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID            City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0  puerto natales -51.7236  -72.4875      3.82        91          90   
1        1      whitehorse  60.7161 -135.0538    -10.57        92         100   
2        2       kununurra -15.7667  128.7333     32.99        43          60   
3        3         ushuaia -54.8000  -68.3000      4.81        81          75   
4        4    puerto aysen -45.4000  -72.7000      4.67        99          94   

   Wind Speed Country        Date  
0        4.34      CL  1701513791  
1        3.09      CA  1701513791  
2        3.09      AU  1701513792  
3        7.20      AR  1701513782  
4        2.38      CL  1701513793

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot

humidity_map = city_data_df.hvplot.points("Lng", "Lat",                                   
                                  geo=True,
                                  tiles='OSM', 
                                  size='Humidity',
                                  xlim=(-180,180),
                                  ylim=(-70,70),
                                  frame_width=700,
                                  frame_height =400,
                                  color='Humidity',
                                  colormap='PuBu',
                                  hover_cols=['City','Country'],
                                  clabel='Humidity %'
                                 )

# Display the map

humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

A max temperature lower than 27 degrees but higher than 21.

Wind speed less than 4.5 m/s.

Zero cloudiness

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df[
    (city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) 
    & (city_data_df['Wind Speed'] < 4.5) 
    & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_cities = ideal_cities.dropna()

# Display sample data
ideal_cities.sample()

City_ID         City     Lat     Lng  Max Temp  Humidity  Cloudiness  \
307      307  tamanrasset  22.785  5.5228     23.95        12           0   

     Wind Speed Country        Date  
307        2.57      DZ  1701513944

### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.sample()

City Country      Lat      Lng  Humidity Hotel Name
356  macae      BR -22.3708 -41.7869        89

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : 'accommodation.hotel',
    'apiKey' : geoapify_key,
    'limit' : 20
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
champerico - nearest hotel: Hotel y Restaurante El Submarino
gwadar - nearest hotel: Sadaf Resorts
remire-montjoly - nearest hotel: Complexe Belova
tamanrasset - nearest hotel: فندق الأمان
matoury - nearest hotel: La Marmotte
macae - nearest hotel: Hotel Plaza
reggane - nearest hotel: No hotel found
aoulef - nearest hotel: دار الضياف


City Country      Lat      Lng  Humidity  \
181       champerico      GT  14.3000 -91.9167        83   
186           gwadar      PK  25.1216  62.3254        57   
220  remire-montjoly      GF   4.9167 -52.2667       100   
307      tamanrasset      DZ  22.7850   5.5228        12   
310          matoury      GF   4.8483 -52.3317       100   
356            macae      BR -22.3708 -41.7869        89   
467          reggane      DZ  26.7158   0.1714         9   
561           aoulef      DZ  26.9667   1.0833         9   

                           Hotel Name  
181  Hotel y Restaurante El Submarino  
186                     Sadaf Resorts  
220                   Complexe Belova  
307                       فندق الأمان  
310                       La Marmotte  
356                       Hotel Plaza  
467                    No hotel found  
561                        دار الضياف

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display


# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles='OSM',
    size='Humidity',
    xlim=(-150, 150),
    ylim=(-70,70),
    frame_width=700,
    frame_height=400,
    color='City',
    colormap='Dark2',
    hover_cols=['City','Hotel Name', 'Country'],  # Adding 'Hotel Name' and 'Country' to hover information
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)